In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls /content/drive/My\ Drive/

 algo  'สำเนาของ 3-1 Final DQA Tools v.0.6_forPrint.xlsx'


In [8]:
!pip install ta

In [9]:

import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go
import ta
from ta import add_all_ta_features
from ta.utils import dropna
template = "plotly_dark"

In [10]:
# Manual Download https://forexsb.com/historical-forex-data -> no volume

In [11]:
profit_color_continuous_scale =["red","orange" ,"White","lightgreen","green"]

In [12]:
## Historical Order Data
raw_df = pd.read_excel('/content/drive/MyDrive/algo/input/ReportHistory-32012857.xlsx',skiprows=6)
raw_df = raw_df[~raw_df['Commission'].isnull()]
time_config = {
     "format":'%Y.%m.%d %H:%M:%S',
     "errors":'coerce'
}
# raw_df['Time.1'] = pd.to_datetime(raw_df['Time.1'], format='%Y.%m.%d %H:%M:%S', errors='coerce')
raw_df["Open Time"] = pd.to_datetime(raw_df["Time"],**time_config)
raw_df["Close Time"] = pd.to_datetime(raw_df["Time.1"],**time_config)
raw_df = raw_df.dropna(subset=['Time.1'])
raw_df["Volume"] = pd.to_numeric(raw_df["Volume"], errors="coerce")

# Step 2: Convert to UTC
# raw_df["Open Time"] = raw_df["Open Time"].dt.tz_localize("Asia/Bangkok")
# raw_df["Close Time"] = raw_df["Close Time"].dt.tz_localize("Asia/Bangkok")
# raw_df["Open Time"] = raw_df["Open Time"].dt.tz_convert("UTC")
# raw_df["Close Time"] = raw_df["Close Time"].dt.tz_convert("UTC")
raw_df["Open Time"] = raw_df["Open Time"] - pd.Timedelta(hours=3)
raw_df["Close Time"] = raw_df["Close Time"] - pd.Timedelta(hours=3)

raw_df["Open Time (15m)"] = raw_df["Open Time"].dt.floor("15min")
order_counts = raw_df.groupby("Open Time (15m)").size().reset_index(name="Orders in 15m")
raw_df = raw_df.merge(order_counts, on="Open Time (15m)", how="left")
# Drop rows where Volume is NaN (invalid)
raw_df = raw_df[raw_df["Volume"].notna()]
raw_df = raw_df.rename(columns={
    "Price": "Open Price",
    "Price.1": "Close Price",
    "Profit": "Profit USD"
})
dtype_mapping = {

    "Profit USD":'int64'
}
raw_df = raw_df.astype(dtype_mapping)
# Drop irrelevant or unnamed columns
df = raw_df[["Open Time (15m)","Orders in 15m","Open Time", "Close Time", "Symbol", "Type", "Volume", "Open Price", "Close Price", "T / P", "Profit USD"]]


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [13]:
df

,Open Time (15m),Orders in 15m,Open Time,Close Time,Symbol,Type,Volume,Open Price,Close Price,T / P,Profit USD
0,2025-03-24 10:15:00,1.0,2025-03-24 10:29:47,2025-03-25 05:23:12,XAUUSDc,buy,1.0,3033.09,3019.35,3036.09,-1374
1,2025-03-24 10:30:00,1.0,2025-03-24 10:31:55,2025-03-25 05:23:12,XAUUSDc,buy,1.0,3030.08,3019.4,3033.08,-1068
2,2025-03-24 11:00:00,1.0,2025-03-24 11:08:14,2025-03-25 05:23:12,XAUUSDc,buy,1.0,3026.99,3019.37,3029.99,-762
3,2025-03-24 13:15:00,1.0,2025-03-24 13:27:47,2025-03-25 05:23:11,XAUUSDc,buy,1.0,3023.97,3019.37,3026.97,-460
4,2025-03-24 14:15:00,2.0,2025-03-24 14:23:20,2025-03-25 05:23:11,XAUUSDc,buy,1.0,3020.9,3019.36,3023.9,-154
...,...,...,...,...,...,...,...,...,...,...,...
2606,2025-04-23 02:45:00,4.0,2025-04-23 02:59:32,2025-04-23 02:59:40,XAUUSDc,buy,2.0,3354.56,3354.63,3357.56,14
2607,2025-04-23 03:00:00,13.0,2025-04-23 03:12:37,2025-04-23 03:19:28,XAUUSDc,buy,2.0,3336.51,3339.51,3339.51,600
2608,2025-04-23 03:00:00,13.0,2025-04-23 03:13:00,2025-04-23 03:14:44,XAUUSDc,buy,2.0,3333.36,3336.36,3336.36,600
2609,2025-04-23 03:00:00,13.0,2025-04-23 03:13:17,2025-04-23 03:13:47,XAUUSDc,buy,2.0,3329.99,3333.29,3332.99,660


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2611 entries, 0 to 2610
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Open Time (15m)  2611 non-null   datetime64[ns]
 1   Orders in 15m    2611 non-null   float64       
 2   Open Time        2611 non-null   datetime64[ns]
 3   Close Time       2611 non-null   datetime64[ns]
 4   Symbol           2611 non-null   object        
 5   Type             2611 non-null   object        
 6   Volume           2611 non-null   float64       
 7   Open Price       2611 non-null   object        
 8   Close Price      2611 non-null   object        
 9   T / P            2610 non-null   object        
 10  Profit USD       2611 non-null   int64         
dtypes: datetime64[ns](3), float64(2), int64(1), object(5)
memory usage: 309.3+ KB


In [15]:
import plotly.express as px


In [16]:
# fig2 = px.scatter(
#     df,
#     x="Open Time",
#     y="Open Price",
#     size="Volume",
#     color="Type",
#     title="Trade Entry Prices",
#     hover_data=["Close Price", "Profit USD"]
# )
# fig2.show()


In [17]:

# Example DataFrame cleanup if not already done
df["Volume"] = pd.to_numeric(df["Volume"], errors="coerce")
df = df[df["Volume"].notna()]

# Melt the dataframe to show both Open and Close price as individual points
df_melted = df.melt(
    id_vars=["Open Time (15m)","Open Time", "Close Time", "Type", "Profit USD"],
    value_vars=["Open Price", "Close Price"],
    var_name="Price Type",
    value_name="Price"
)

# Use Open Time for both points to align them visually
df_melted["Time"] = df_melted.apply(
    lambda row: row["Open Time"] if row["Price Type"] == "Open Price" else row["Close Time"],
    axis=1
)

# Color profit: positive = green, negative = red
df_melted["Profit Color"] = df_melted["Profit USD"].apply(
    lambda x: "green" if x >= 0 else "red")
df_melted["Profit Size"] = df_melted["Profit USD"].apply(
    lambda x: abs(x))

fig = px.scatter(
    df_melted,
    x="Time",
    y="Price",
    color="Profit USD",
    # color_continuous_scale='YlGn',
    color_continuous_midpoint=0,  # This centers the color scale at 0
    range_color=[-300, 300],           # Set min/max of color range
    # size="Profit Size",
    color_continuous_scale=profit_color_continuous_scale,
    symbol="Type",
    facet_col="Type",
    title="Entry and Exit Prices by Trade",
    hover_data=["Profit USD", "Type"]
)

fig.update_layout(height=500, xaxis_title="Time", yaxis_title="Price",template=template)
fig.show()

<ipython-input-17-ebc2b8d15928>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Volume"] = pd.to_numeric(df["Volume"], errors="coerce")


In [18]:
df_melted

,Open Time (15m),Open Time,Close Time,Type,Profit USD,Price Type,Price,Time,Profit Color,Profit Size
0,2025-03-24 10:15:00,2025-03-24 10:29:47,2025-03-25 05:23:12,buy,-1374,Open Price,3033.09,2025-03-24 10:29:47,red,1374
1,2025-03-24 10:30:00,2025-03-24 10:31:55,2025-03-25 05:23:12,buy,-1068,Open Price,3030.08,2025-03-24 10:31:55,red,1068
2,2025-03-24 11:00:00,2025-03-24 11:08:14,2025-03-25 05:23:12,buy,-762,Open Price,3026.99,2025-03-24 11:08:14,red,762
3,2025-03-24 13:15:00,2025-03-24 13:27:47,2025-03-25 05:23:11,buy,-460,Open Price,3023.97,2025-03-24 13:27:47,red,460
4,2025-03-24 14:15:00,2025-03-24 14:23:20,2025-03-25 05:23:11,buy,-154,Open Price,3020.9,2025-03-24 14:23:20,red,154
...,...,...,...,...,...,...,...,...,...,...
5217,2025-04-23 02:45:00,2025-04-23 02:59:32,2025-04-23 02:59:40,buy,14,Close Price,3354.63,2025-04-23 02:59:40,green,14
5218,2025-04-23 03:00:00,2025-04-23 03:12:37,2025-04-23 03:19:28,buy,600,Close Price,3339.51,2025-04-23 03:19:28,green,600
5219,2025-04-23 03:00:00,2025-04-23 03:13:00,2025-04-23 03:14:44,buy,600,Close Price,3336.36,2025-04-23 03:14:44,green,600
5220,2025-04-23 03:00:00,2025-04-23 03:13:17,2025-04-23 03:13:47,buy,660,Close Price,3333.29,2025-04-23 03:13:47,green,660


In [19]:
hist_cols = ['datetime', 'open', 'high', 'low', 'close', 'volume']

hist_df = pd.read_csv('/content/drive/MyDrive/algo/input/XAUUSD_M1.csv',names=hist_cols)

In [20]:
hist_df

,datetime,open,high,low,close,volume
0,2025-01-16 03:57,2696.848,2697.148,2696.838,2697.025,1
1,2025-01-16 03:58,2697.005,2697.035,2696.838,2696.869,1
2,2025-01-16 03:59,2696.838,2697.139,2696.838,2696.955,1
3,2025-01-16 04:00,2696.929,2697.315,2696.838,2697.065,1
4,2025-01-16 04:01,2697.115,2697.199,2696.865,2697.065,1
...,...,...,...,...,...,...
99995,2025-04-30 02:55,3307.175,3307.765,3306.585,3306.885,1
99996,2025-04-30 02:56,3306.945,3307.445,3306.515,3306.915,1
99997,2025-04-30 02:57,3306.935,3307.205,3304.635,3305.505,1
99998,2025-04-30 02:58,3305.585,3306.555,3305.105,3305.985,1


In [21]:

# Step 2: Prepare datetime
hist_df['datetime'] = pd.to_datetime(hist_df['datetime'])

hist_df = hist_df[hist_df['datetime']>='2025-04-01']
hist_df = hist_df[hist_df['datetime']<'2025-04-24']

# Step 3: Plotting
fig = go.Figure(data=[go.Candlestick(
    x=hist_df['datetime'],
    open=hist_df['open'],
    high=hist_df['high'],
    low=hist_df['low'],
    close=hist_df['close'],
    increasing_line_color='green',
    decreasing_line_color='red'
)])

fig.update_layout(
    title='Gold Spot Price (15min intervals)',
    xaxis_title='Time',
    yaxis_title='Price (USD)',
    xaxis_rangeslider_visible=False,
    template=template
)

fig.show()

In [22]:
ta_df = dropna(hist_df)
ta_df['datetime'] = pd.to_datetime(ta_df['datetime'], errors='coerce')
ta_df = ta_df.dropna(subset=['datetime'])

# Add ta features filling NaN values
ta_df = add_all_ta_features(
    ta_df, open="open", high="high", low="low", close="close", volume="volume", fillna=True)

In [23]:
for period in [10, 20, 50, 100, 200]:
    ta_df[f'ema_{period}'] = ta.trend.EMAIndicator(close=ta_df['close'], window=period, fillna=True).ema_indicator()

# Add RSI for common periods
for period in [14, 21]:
    ta_df[f'rsi_{period}'] = ta.momentum.RSIIndicator(close=ta_df['close'], window=period, fillna=True).rsi()

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Create subplots
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True,
    vertical_spacing=0.1,
    row_heights=[0.7, 0.3],
    subplot_titles=("Price with EMAs (OHLC) and Trades", "RSI")
)

# --- OHLC Chart ---
fig.add_trace(go.Candlestick(
    x=ta_df['datetime'],
    open=ta_df['open'],
    high=ta_df['high'],
    low=ta_df['low'],
    close=ta_df['close'],
    # increasing_line_color='green',
    # decreasing_line_color='red',

), row=1, col=1)
fig.update_layout(xaxis_rangeslider_visible=False)
# --- EMA Lines ---
for period in [50, 200]:
    fig.add_trace(go.Scatter(
        x=ta_df['datetime'],
        y=ta_df[f'ema_{period}'],
        mode='lines',
        name=f'EMA {period}',
        connectgaps=False,
    ), row=1, col=1)

# --- Custom Trade Markers: "buy" and "sell" ---
for trade_type, symbol in [("buy", "arrow-up"), ("sell", "arrow-down")]:
    trade_subset = df_melted[df_melted["Type"].str.lower() == trade_type]

    fig.add_trace(go.Scatter(
        x=trade_subset["Time"],
                # x=trade_subset["Open Time (15m)"],
        y=trade_subset["Price"],
        mode="markers",
        name=f"{trade_type.title()} Trade",
        connectgaps=False,
        marker=dict(
            symbol=symbol,
            size=12,
            color=trade_subset["Profit USD"],  # Color by profit
            colorscale=profit_color_continuous_scale,  # Optional custom scale
            cmin=-300,  # Set these based on your profit range
            cmax=300,
            colorbar=dict(title="Profit USD"),
            line=dict(width=1, color='black')
        ),
        hovertemplate=(
            "Time: %{x}<br>" +
            "Price: %{y}<br>" +
            "Profit USD: %{customdata[0]}<br>" +
            "Type: %{customdata[1]}"
        ),
        customdata=trade_subset[["Profit USD", "Type"]].values
    ), row=1, col=1)


# --- RSI Line ---
fig.add_trace(go.Scatter(
        x=ta_df['datetime'],
    y=ta_df['rsi_14'],
    mode='lines',
    name='RSI 14',
    line=dict(color='orange'),
    connectgaps=False,
), row=2, col=1)

# # RSI Overbought/Oversold Lines
start_date = ta_df['datetime'].iloc[0]
end_date = ta_df['datetime'].iloc[-1]

fig.add_shape(type="line", x0=start_date, x1=end_date, y0=70, y1=70,
              line=dict(dash='dash', color='red'), row=2, col=1)
fig.add_shape(type="line", x0=start_date, x1=end_date, y0=30, y1=30,
              line=dict(dash='dash', color='green'), row=2, col=1)
overbought = ta_df[ta_df['rsi_14'] > 70]
oversold = ta_df[ta_df['rsi_14'] < 30]

# Shade overbought regions on main chart
for dt in overbought['datetime']:
    fig.add_vrect(
        x0=dt, x1=dt,
        fillcolor="red", opacity=0.2,
        layer="below", line_width=0,
        row=1, col=1
    )

# Shade oversold regions on main chart
for dt in oversold['datetime']:
    fig.add_vrect(
        x0=dt, x1=dt,
        fillcolor="green", opacity=0.2,
        layer="below", line_width=0,
        row=1, col=1
    )
# Layout Settings
fig.update_layout(
    title='Stock OHLC with EMA, Trade Entries (Buy/Sell), and RSI',
    height=800,
    legend=dict(x=0, y=1.1, orientation='h'),
    xaxis2_title='Date',
    yaxis1_title='Price',
    yaxis2_title='RSI',
    template=template
)

fig.show()


In [ ]:
ta_df[ta_df['datetime'].isna()]

In [ ]:
df_melted[df_melted['Time'].isna()]